## 1. Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

## 2. Get articles

In [ ]:
'''
urls = [("https://www.national-geographic.pl/nauka?page=", 314), ("https://www.national-geographic.pl/ludzie?page=", 97),
        ("https://www.national-geographic.pl/przyroda?page=", 87), ("https://www.national-geographic.pl/historia?page=", 34), 
        ("https://www.national-geographic.pl/odkrycia?page=", 35), ("https://www.national-geographic.pl/kosmos?page=", 29)]

'''

In [ ]:
urls = [("https://www.national-geographic.pl/nauka?page=", 10), ("https://www.national-geographic.pl/ludzie?page=", 10),
        ("https://www.national-geographic.pl/przyroda?page=", 10), ("https://www.national-geographic.pl/historia?page=", 10), 
        ("https://www.national-geographic.pl/odkrycia?page=", 10), ("https://www.national-geographic.pl/kosmos?page=", 10)]

In [ ]:
coverpage_news = []
for (url, nr_pages) in urls:
  for nr in range(1, nr_pages+1):
    r1 = requests.get(url+str(nr))
    r1.status_code
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news += soup1.find_all('a', class_='d-flex flex-column type-content-')
    
len(coverpage_news)

1199

In [ ]:
coverpage_news = list(set(coverpage_news))
len(coverpage_news)

882

## 3. Get data

In [ ]:
number_of_articles = len(coverpage_news)
news_contents = []
list_links = []
list_titles = []
list_authors = []
list_categories = []

for n in np.arange(0, number_of_articles):
    # Get link
    link = 'https://www.national-geographic.pl' +str(coverpage_news[n]['href'])
    list_links.append(link)
    print(link)
    
    # Get title
    title = coverpage_news[n].find('h2').get_text()
    list_titles.append(title)
    
    # Get content
    article = requests.get(link)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('div', class_='article-body')
    x = body[0].find_all('p')   
    # Unifying paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)-2):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)        
    news_contents.append(final_article)

    # Get author
    if(soup_article.find('span', class_='author-item')):
      author = soup_article.find('span', class_='author-item').get_text()
    else:
      author = 'Unknown'
    list_authors.append(author)

    # Get category
    basic_category = str(article_content,'utf-8').split('contentSubCategory":"')[1]
    basic_category = basic_category.split('","articleExtraCategories"')[0]    
    try:
      extra_category = str(article_content,'utf-8').split('articleExtraCategories":"National Geographic > ')[1]
      extra_category = extra_category.split('","articleCategory"')[0]
      category = [basic_category, extra_category]
    except IndexError:
      category = [basic_category]

    list_categories.append(category)



https://www.national-geographic.pl/artykul/nie-wszystkie-slimaki-wygladaja-tak-samo-oto-niebieski-smok-co-o-nim-wiemy-220411010943
https://www.national-geographic.pl/artykul/zycie-na-ziemi-pojawilo-sie-o-wiele-wczesniej-niz-sadzili-naukowcy-kiedy-powstalo-220414121408
https://www.national-geographic.pl/artykul/noc-spadajacych-gwiazd-kiedy-na-niebie-widac-najwiecej-meteorow-jak-obserwowac-spadajace-gwiazdy
https://www.national-geographic.pl/artykul/dzis-w-birmie-to-jeden-wielki-znak-zapytania-o-zyciu-w-tym-fascynujacym-kraju-opowiada-aleksandra-minkiewicz-dyrektorka-yangon-film-school
https://www.national-geographic.pl/artykul/co-chrumkaja-swinie-naukowcy-po-raz-pierwszy-odszyfrowali-ich-emocje-220308124739
https://www.national-geographic.pl/artykul/w-indyjskim-miescie-wprowadzono-zakaz-sprzedazy-i-konsumpcji-miesa
https://www.national-geographic.pl/artykul/filmy-o-zjawiskach-przyrody-burzach-tornadach-powodziach-tajfunach-trzesieniach-ziemi-zorza-polarnych-ktore-musisz-zobaczyc-oto-top

In [ ]:
len(news_contents)

882

## 4. Create DataFrame

In [ ]:
df_articles = pd.DataFrame(
    {'Title': list_titles,
     'Link': list_links,
     'Author': list_authors,
     'Category': list_categories,
     'Content': news_contents 
     })

In [ ]:
df_articles

,Title,Link,Author,Category,Content
0,Nie wszystkie ślimaki wyglądają tak samo. Oto ...,https://www.national-geographic.pl/artykul/nie...,Unknown,[Przyroda],Morza i oceany często są porównywane do przest...
1,"Życie na Ziemi pojawiło się o wiele wcześniej,...",https://www.national-geographic.pl/artykul/zyc...,Unknown,"[Odkrycia, Nauka]","Ziemia powstała z mgławicy słonecznej ok. 4,5 ..."
2,Noc spadających gwiazd. Kiedy na niebie widać ...,https://www.national-geographic.pl/artykul/noc...,Unknown,[Kosmos],Meteoroidy to krążące po kosmosie kawałki skał...
3,„Dziś” w Birmie to jeden wielki znak zapytania...,https://www.national-geographic.pl/artykul/dzi...,"Patronat nad festiwalem „Pięć Smaków""",[Ludzie],W programie 15. edycji znalazła się m.in. sekc...
4,Co chrumkają świnie? Naukowcy po raz pierwszy ...,https://www.national-geographic.pl/artykul/co-...,Unknown,[Nauka],"Wszyscy dobrze wiemy, jakie odgłosy wydają świ..."
...,...,...,...,...,...
877,Pracujesz nad mięśniami? Nie tylko będziesz si...,https://www.national-geographic.pl/artykul/pra...,Unknown,"[Nauka, Odkrycia]","Wiadomości, jakie mają dla nas badacze, są zas..."
878,"Egzoplanety, czyli planety pozasłoneczne. To o...",https://www.national-geographic.pl/artykul/egz...,Unknown,[Kosmos],W języku greckim „exo” oznacza zewnętrzny. Dla...
879,Nasza galaktyka jest prawdopodobnie pełna mart...,https://www.national-geographic.pl/artykul/nas...,Unknown,[Kosmos],"Nowe badanie, którego autorami są trzej fizycy..."
880,4 zagadki Wszechświata. Czy zapomniana teoria ...,https://www.national-geographic.pl/artykul/skr...,Michał Rolecki,"[Nauka, Kosmos]",Ogólna teoria względności opracowana przez Alb...


In [ ]:
len(df_articles)

882

In [ ]:
df_articles.to_csv(r'data.csv', index=False)

In [ ]:
news_contents[0]

'Morza i oceany często są porównywane do przestrzeni kosmicznej. Nie ze względu na ciemności w najgłębszych partiach, a przez to, że niewiele wiemy o większości zamieszkujących je stworzeń. Autorzy badań opublikowanych na łamach czasopisma naukowego „Science Advances” twierdzą, że ponad połowa form życia w oceanach jest niezbadana. Niektóre z gatunków\xa0znanych nauce występują tak rzadko, że ich zaobserwowanie w środowisku naturalnym może wywołać niemałą sensacją. Tak też było w Stanach Zjednoczonych. Ślimaki kojarzą nam się z obślizgłymi stworzonkami, które chowają się w muszli. Rzeczywiście, większość gatunków występujących w Polsce można tak opisać. Całą gromadę tworzy jednak ponad 100 tys. gatunków ślimaków lądowych i morskich. Niektóre z nich mają bardzo nietypowy wygląd. Przekonał się o tym Erick Yanta, Amerykanin wypoczywający razem z żoną na wyspie Mustang w Zatoce Meksykańskiej. \nNiebieski smok nie przypomina ślimaków, jakie spotykamy w Polsce. / fot. James Chasty Para space